In [1]:
import os
os.environ["nnUNet_raw"] = "/mnt/d/nnunet/raw"
os.environ["nnUNet_preprocessed"] = "/mnt/d/nnunet/preprocessed"
os.environ["nnUNet_results"] = "/mnt/d/nnunet/results"

from nnunetv2.paths import nnUNet_preprocessed, nnUNet_results, nnUNet_raw

print(nnUNet_preprocessed, nnUNet_results, nnUNet_raw)

/mnt/d/nnunet/preprocessed /mnt/d/nnunet/results /mnt/d/nnunet/raw


In [6]:
import os
import torch
import shutil
from nnunet_api import NnUnetApi
from tools.data_reformat import data_prepare
from tools.json_pickle_stuff import copy_plans_json

# =================================================================================
# 1. Configuration
# =================================================================================
# --- Define the IDs and paths for your datasets and models ---

# ABS path to the raw data directory: each subject has its own folder with BraTS namind convention
RAW_DATA_PATH = "/mnt/d/Datasets/mehdi_code_test/sample_data" #"PATH_TO_SAMPLE_RAW_DATA"

# The Dataset ID of the model you want to use for pre-training.
# This model's architecture and plans will be transferred to your new dataset.
PRETRAINED_DATASET_ID = 770

# The Dataset ID of your new dataset that you want to fine-tune on.
# Make sure you have already converted this dataset to the nnU-Net format.
FINETUNE_DATASET_ID = 666

# An identifier for the new plans that will be created for your fine-tuning dataset.
# It's good practice to give it a descriptive name.
FINETUNE_PLANS_ID = 'nnUNetPlans_finetune_from_brats'

# The full path to the pre-trained model's checkpoint file.
# This file contains the weights that will be used to initialize your new model.
PRETRAINED_CHECKPOINT_PATH = os.path.join(nnUNet_results, "Dataset770_BraTSGLIPreCropRegion/nnUNetTrainer__nnUNetResEncUNetPlans__3d_fullres/fold_0/checkpoint_final.pth")

# The GPU device to use for training.
# DEVICE = torch.device('cuda')

# Fold number you want to train (on the fine tunning dataset)
FOLD = 0

# NUMBER of epochs to train the model
N_EPOCHS = 3

# Initial Learning Rate for model training
INIT_LR = 1e-3

# ABS Path to the folder where testing data is located
INPUT_FOLDER_INFER = "/mnt/d/Datasets/mehdi_code_test/sample_test_decathlon" #'PATH_TO_TESTING_DATA'

# ABS Path to the folder where the results of inference will be saved
OUTPUT_FOLDER_INFER_FINETUNE = './test_ftune' #'PATH_TO_SAVE_RESULTS_FINETUNE'
OUTPUT_FOLDER_INFER_PRETRAINED = './test_pretrain' #'PATH_TO_SAVE_RESULTS_PRETRAINED'


In [7]:
# ## --- Step 00: Preparing the raw dataset into Decathlon format
print("\nStep 0: Running Data Preparation script...")
DST_DATA_NAME = "Dataset"+str(FINETUNE_DATASET_ID)+"_finetune_Decathlon"
data_prepare(RAW_DATA_PATH, os.path.join(nnUNet_raw, DST_DATA_NAME))
n_case=len(os.listdir(os.path.join(nnUNet_raw, DST_DATA_NAME,"labelsTr")))
copy_plans_json("./dataset.json", os.path.join(nnUNet_raw, DST_DATA_NAME), n_case)



Step 0: Running Data Preparation script...
--------
data reformat in process for case 1 out of 10 ...
data reformat in process for case 2 out of 10 ...
data reformat in process for case 3 out of 10 ...
data reformat in process for case 4 out of 10 ...
data reformat in process for case 5 out of 10 ...
data reformat in process for case 6 out of 10 ...
data reformat in process for case 7 out of 10 ...
data reformat in process for case 8 out of 10 ...
data reformat in process for case 9 out of 10 ...
data reformat in process for case 10 out of 10 ...
--------
All files were reformated, ready for segmentation!


In [9]:

# =================================================================================
# 2. Initialize the API
# =================================================================================
api = NnUnetApi()
## --- Step 1: Extract the Fingerprint for Your New Dataset ---
## This step analyzes the properties of your new dataset (image sizes, spacings, etc.)
## and creates a "fingerprint" file. This is a prerequisite for any planning.
## This wraps: nnUNetv2_extract_fingerprint CLI
print(f"Step 1: Extracting fingerprint for Dataset {FINETUNE_DATASET_ID}...")
api.extract_fingerprint(
    finetune_dataset_id=FINETUNE_DATASET_ID
)
print("-> Fingerprint extracted.\n")

Step 1: Extracting fingerprint for Dataset 666...
Dataset666_finetune_Decathlon
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer


100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.12it/s]


-> Fingerprint extracted.



In [10]:
# --- Step 2: Load and Apply the Pre-trained Model's Plans ---
# This is the key step for aligning architectures. It takes the plans
# (network topology, patch size, normalization, etc.) from the pre-trained
# model and applies them to your new dataset's fingerprint, creating a new
# plan file specifically for fine-tuning.
# This wraps: nnUNetv2_move_plans_between_datasets CLI
print(f"Step 2: Applying plans from Dataset {PRETRAINED_DATASET_ID} to Dataset {FINETUNE_DATASET_ID}...")
api.apply_pretrained_plans(
    pretrained_dataset_id=PRETRAINED_DATASET_ID,
    finetune_dataset_id=FINETUNE_DATASET_ID,
    finetune_plans_identifier=FINETUNE_PLANS_ID
)
print(f"-> Plans applied. New plans identifier is '{FINETUNE_PLANS_ID}'.\n")


Step 2: Applying plans from Dataset 770 to Dataset 666...


AssertionError: Cannot move plans because preprocessed directory of source dataset is missing. Run nnUNetv2_plan_and_preprocess for source dataset first!